# SAR Flood Mapping

Before we start working with SAR Imagery and Google Earth Engine (GEE), we first need to import a few libraries and perform authentication using your GEE account. 

Note, if this is your first time using GEE, you must sign up for an Earth Engine Developer Account using [this link](https://earthengine.google.com/new_signup/). It usually takes 2-3 business days, following which you will be able to use all GEE APIs in Python (and by extension in the code editor online)

You will also need to import Folium, an interactive mapping application that meshes nicely with Earth Engine, and provides direct methods for much GEE's JavaScript functionality

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

So let's get started, first we need to specify two sets of date ranges. 
The first defines a period of time you know that no flooding had occurred, and we call this ```before_start``` and ```before_end```. We also need some date range for potentially flooded imagery, we use ```after_start``` and ```after_end``` to define another range, which we use to compare against our reference images

In [ ]:
before_start= '2019-03-01';
before_end='2019-03-10';

after_start='2021-08-28';
after_end='2021-09-05';

### Polarization and Wave Stuff  
Crash course on transverse waves and polarization angles:  
    Polarization refers to the perpendicular restriction to the oscillations of transverse waves.

Horizontal vs Vertical Polarisation
Horizontally polarised waves are those which are restricted in the vertical domain (with respect to the wave generator/observer). In other terms, the transverse wave is free to oscillate left-to-right-to-left but not up-down

Vertically polarised waves are allowed to oscillate in the up-down manner, but not in the side-to-side manner.

What Does This Have to do with SAR?
SAR Imagery **depends** on radio-waves. The 'picture' taken by a SAR camera is as a result of reflections from objects of incident radio waves. If you screamimg in a large hall is a SAR Satellite emitting its radio-waves, the image it takes is analogous to the echo you receive from your voice bouncing around as it is reflected back to your ears.

The data we will use for this tutorial is sourced from the Sentinel-1 platform, which operates in the C-band (around 1-2GHz) in either single or dual polarizations.

When we talk about polarizations that Sentine-1 operates in, we must define a sending and receiving mode. For example, *VV* indicates that the SAR imager is transmitting a vertically-polarized radiowave, and is listening in the same vertical polarization. If however, Sentinel-1 operates in *VH* mode, it is then transmitting in the vertical orientation, whilst receiving in the *perpendicular* orientation. This may seem counter-intuitive at first, however, certain objects (as we'd see shortly), cause certain types of scattering which can be observed by the disappearance (or appearance) of the *opposite* type of polarization.

For example, **surface-scattering** occurs when radiowaves encounter a rough surface (such as bare soil), in this case, the VV and HH polarization tend to "see" these surfaces more clearly, as the complete redirection of incident waves is far less likely.

Conversely, **volume scattering** occurs when multiple rebounces and redirection of incident waves occur, such as within the thick canopy of dense forests. For This application, the HV polarization tends to be the highest. 

In between both extremes, **double-reflection** between perpendicular or orthogonal surfaces will also result in a high return signal for the *HH* polarization.

According to (Carreño Conde, Francisco, and María De Mata Muñoz. 2019. "Flood Monitoring Based on the Study of Sentinel-1 SAR Images: The Ebro River Case Study" Water 11, no. 12: 2454. https://doi.org/10.3390/w11122454), the VH polarization (send in vertical, receive in horizontal) yields the most consistent results in flood mapping. A word of caution however, since SAR wave reflection is also directly influence by dielectric properties of the reflecting body, you may need to experiment with different polarizations in order to achieve the desired results.



In [ ]:
polarization = "VH"

Sentinel-1 consists two satellites orbiting on opposite sides of the earth at any given point in time (Sentinel-1A and -1B). For this reason, for any patch on the earth's surface, any one of these satellites may be travelling upward (towards the north pole) or downwards (towards the south pole) depending on where in the orbital cycle each satellite is at any given point in time.

Since the *direction* of pass can affect radiometric corrections required by any orbital-based imager, we wish to keep the direction of pass consistent for any images we wish to compare side-by-side

In [ ]:
pass_direction = "ASCENDING"

At this point,we define a polygon (by its endpoints) encompassing the New Orleans area. For sake of ease-of-intuition, I have also included a true-color image of the area being encompassed 

In [ ]:
geometry = ee.Geometry.Polygon(
        [[[-92.93870870124604, 30.57399079248121],
          [-92.93870870124604, 29.1256766111735],
          [-88.87376729499607, 29.1256766111735],
          [-88.87376729499607, 30.57399079248121]]], None, True);

We create a ```FeatureCollection``` from our geometry, and name it aoi (for area-of-interest). A FeatureCollection is a wrapper that can handle multiple other types and enables functionality on the wrapped elements, such as sorting, filtering and rendering

In [ ]:
aoi = ee.FeatureCollection(geometry);

## Data Selection
In the following piece of code, we select the Sentinel-1 dataset and filter it to obtain our required bands/combinations as specified above.

In [ ]:
collection= ee.ImageCollection('COPERNICUS/S1_GRD')\
  .filter(ee.Filter.eq('instrumentMode','IW'))\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))\
  .filter(ee.Filter.eq('orbitProperties_pass',pass_direction))\
  .filter(ee.Filter.eq('resolution_meters',10))\
  .filterBounds(aoi)\
  .select(polarization)\


 It's a bit to take in so let's break it down:  
 First, we select Sentinel-1



In [ ]:
ee.ImageCollection('COPERNICUS/S1_GRD')\

We then filter this ImageCollection, which points to every single image that has ever been captured by Sentinel-1 and uploaded to GEE. We wish to obtain the Interferometric-Wide-Swath Mode of operation (otherwise known as the normal, usual mode for Sentinel-1, see [here](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-1-sar/acquisition-modes) for a breakdown of the different operating modes of Sentinel-1).

In [ ]:
    .filter(ee.Filter.eq('instrumentMode','IW'))\

We then filter (based on the discussion above) to obtain the intended polarizations and pass directions. In this step we also filter by resolution, in order to make our before and after imagery directly comparable.

In [ ]:
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))\
    .filter(ee.Filter.eq('orbitProperties_pass',pass_direction))\
    .filter(ee.Filter.eq('resolution_meters',10))\

Finally, we filter our resulting images by the area-of-interest defined in our ```FeatureCollection``` above, and select the required polarization.

In [ ]:
    .filterBounds(aoi)
    .select(polarization)

Now we wish to further zero-in on images from before and after/during potential flood events. To do this, we use the filtered ```collection``` from above, and use ```filterDate``` to get two sets of image collections

In [ ]:
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start,after_end)

before = before_collection.mosaic().clip(aoi);
after = after_collection.mosaic().clip(aoi);

### Despeckle

This step is especially important for radar-based imagery. *Speckle* is the term given to salt-and-pepper noise which is caused by out-of-phase waves reflected from the given target (in this case the patch of earth under current observation). We can use Earth Enginer's inbuilt functions to filter or *smooth* the filtered images using a defined smoothing radius.

This is a morphological operation taking into account the mean values of all pixels within a radius of 50

In [ ]:
smoothing_radius = 50;
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters');
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters');


## Visualizing Flood Pixels

Now we take both our completely pre-processed sets of images, and find the difference between them. To do this, we *divide* the after images by the before and threshold the result. The intuition is that, if any area in the after image has changed significantly, and given that we are working in the *VH* polarization (observing potential floods), we want to highlight the areas that have experienced the most change.

In [ ]:
difference = after_filtered.divide(before_filtered);

difference_threshold = 1.25
threshold = difference_threshold;

Using this differenced image, we can generate a binary image mask using the ```gt()``` function in Earth Engine

In [ ]:
difference_binary = difference.gt(threshold);


## Optional - Further Masking
In this section we use a curated dataset from the ESA (also part of the Copernicus programme). We create a mask using image pixels where water is present for more than 10 months of the year. We do not want to classify inland water bodies (lakes, ravines, etc) as flood



In [ ]:
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality');
swater_mask = swater.gte(10).updateMask(swater.gte(10));


Similar to the above, we create a binary mask for image areas that do contain water for most of the year, and update our initial flood mask from the Sentinel-1 data.

In [ ]:
flooded_mask = difference_binary.where(swater_mask,0);
flooded = flooded_mask.updateMask(flooded_mask);

As a semif-final processing step, we also eliminate spurios pixels, by ensuring that all flooded areas are connected to at least eight other pixels (this value can change, experiment to see what works for you). We use this connected pixel count to again create a binary mask, and update our flooded areas to exclude the pixels that existed in isolation.

In [ ]:
connections = flooded.connectedPixelCount();    
flooded = flooded.updateMask(connections.gte(8));


Finally, we use a Digital Elevation map (or **DEM**), to create the binary mask which removes area of greater than 5% slope. Once again, this value is not hard-and-fast; the intuition is that areas with such a steep slope may not be flooded unless significantly below sea level.

In [ ]:
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM');
terrain = ee.Algorithms.Terrain(DEM);
slope = terrain.select('slope');
flooded = flooded.updateMask(slope.lt(5));

## Visualization

We will be using ```Folium```, which is an interactive library for creating maps in Python. This first section overrides an existing method which allows us to add raster images (and image collections) directly to a Folium map. This function is taken directly from the Earth Engine [developer documentation](https://developers.google.com/earth-engine/guides/python_install#folium-interactive-map-display).

In [ ]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


We create a ```Map``` object using Folium, and add both the before and after images. 

The map objects ```{'min':-25,'max':0}``` set a range of values which would be displayed from the radio images. Because I did not explicitly define a colorscale, pixels with values -25 and below will clip to black, and pixels with values of 0 and greater will clip to white.

### Dangerous Detour
How do we have negative values?! 

Radio imagery is radiometrically corrected and the natural logarithm of the absolute values are taken before any sort of inference or interpretaion. This is due to the wide range of possible pixel values, and is typically manually performed using software such as the Sentinel toolbox. 

In [ ]:
Map = folium.Map()

Map.add_ee_layer(before_filtered, {'min':-25,'max':0},0);
Map.add_ee_layer(after_filtered, {'min':-25,'max':0},1);


Finally, we add the actual flooded area layer, and give it a colour blue using the corresponding hex code.

In [ ]:
Map.add_ee_layer(flooded,{'palette':"0000FF"},'Flooded areas')

display(Map)

### References

*Contains modified Copernicus Sentinel data 2021' for Sentinel data*


[UN Recommended Practice: Radar-based Flood Mapping
](https://www.un-spider.org/advisory-support/recommended-practices/recommended-practice-radar-based-flood-mapping)

K. Dasari, L. Anjaneyulu, P. V. Jayasri and A. V. V. Prasad, "Importance of speckle filtering in image classification of SAR data," 2015 International Conference on Microwave, Optical and Communication Engineering (ICMOCE), 2015, pp. 349-352, doi: 10.1109/ICMOCE.2015.7489764.


Gorelick, Noel, Matt Hancher, Mike Dixon, Simon Ilyushchenko, David Thau, and Rebecca Moore. "Google Earth Engine: Planetary-scale geospatial analysis for everyone." Remote sensing of Environment 202 (2017): 18-27.
